In [398]:
%load_ext autoreload
%autoreload 2

import logging
from urllib.parse import urljoin
from io import StringIO
from typing import Sequence
from datetime import date, datetime

import requests
import pandas as pd
import numpy as np

from etf_scraper.base import SecurityListing
from etf_scraper.scrapers import ISharesListings, SSGAListings, InvescoListings, VanguardListings

logger = logging.getLogger(__name__)

il = InvescoListings.retrieve_listings()
van = VanguardListings.retrieve_listings()
ssga_listings = SSGAListings.retrieve_listings()        
ishares_listings = ISharesListings.retrieve_listings()

#TODO: Vanguard, Invesco - get proof of concept for one of ishares, ssga holdings first

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ].apply(parse_date)
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:318: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  )
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:113: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newva

In [410]:
all_listings = pd.concat([il, van, ssga_listings, ishares_listings])
all_listings.to_csv('./listings_2023_01_15.csv', index=False)

In [401]:
all_listings.isna().sum()

fund_name           0
ticker              0
inception_date      0
benchmark         897
cusip              69
isin              422
exchange          853
item_url          994
fund_type           0
provider            0
product_id        456
asset_class       315
subasset_class    456
product_url       246
region            662
net_assets        662
country           803
dtype: int64

In [393]:
il.tail(3)

,fund_name,ticker,inception_date,benchmark,cusip,isin,exchange,item_url,fund_type,provider
243,Invesco Ultra Short Duration ETF,GSY,2008-02-12,NaN,46090A887,US46090A8870,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco
244,Invesco VRDO Tax-Free ETF,PVI,2007-11-15,PVIX,46138G862,US46138G8621,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco
245,Invesco Variable Rate Investment Grade ETF,VRIG,2016-09-22,NaN,46090A879,US46090A8797,NASDAQ,https://www.invesco.com/us/financial-products/...,ETF,Invesco


In [394]:
ssga_listings.tail(3)

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
207,The Real Estate Select Sector SPDR® Fund,XLRE,https://www.ssga.com/us/en/intermediary/etfs/f...,2015-10-07,US,5.356750e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,3.930098e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA
209,The Utilities Select Sector SPDR® Fund,XLU,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.658127e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y886,US81369Y8865,Utilities Select Sector Index,SSGA


In [395]:
van.tail(3)

,ticker,cusip,fund_name,inception_date,product_id,asset_class,subasset_class,fund_type,product_url,provider
344,VTWAX,922042619,Vanguard Total World Stock Index Fund Admiral ...,2019-02-07,5028,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
345,VTWIX,922042759,Vanguard Total World Stock Index Fund Institut...,2008-10-09,0826,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
346,VCMDX,921939708,Vanguard Commodity Strategy Fund Admiral Shares,2019-06-25,0517,Stock Funds,Specialty,MF,https://advisors.vanguard.com/investments/prod...,Vanguard


In [396]:
ishares_listings.tail(3)

,fund_name,inception_date,ticker,cusip,isin,asset_class,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider
434,iShares ESG Advanced MSCI EAFE ETF,2020-06-16,DMXF,46436E759,US46436E7590,Equity,Large Cap,Broad,Global,https://www.ishares.com/us/products/314362/ish...,314362,4.246446e+08,ETF,IShares
435,iShares MSCI Brazil ETF,2000-07-10,EWZ,464286400,US4642864007,Equity,Large/Mid Cap,Brazil,Latin America,https://www.ishares.com/us/products/239612/ish...,239612,4.972791e+09,ETF,IShares
436,iShares Municipal Bond Index Fund,2018-11-16,BIDPX,066923210,US0669232102,Fixed Income,Municipals,United States,North America,https://www.ishares.com/us/products/306171/ish...,306171,2.273509e+08,MF,IShares


In [378]:
# with open("test_SSGA_SPY_resp.xlsx", 'wb') as f:
#     f.write(spy.content)

spy = SSGAListings.retrieve_holdings_("SPY")
spy

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/utils.py:51: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col] = pd.to_numeric(df[col])


,name,ticker,cusip,sedol,weight,Sector,amount,local_currency,fund_ticker,as_of_date
4,Apple Inc.,AAPL,03783310,2046251,5.981867,Information Technology,166167790.0,USD,SPY,2023-01-12
5,Microsoft Corporation,MSFT,59491810,2588173,5.331199,Information Technology,82835550.0,USD,SPY,2023-01-12
6,Amazon.com Inc.,AMZN,02313510,2000019,2.535409,Consumer Discretionary,98625770.0,USD,SPY,2023-01-12
7,Berkshire Hathaway Inc. Class B,BRK.B,08467070,2073390,1.722998,Financials,20021100.0,USD,SPY,2023-01-12
8,Alphabet Inc. Class A,GOOGL,02079K30,BYVY8G0,1.632137,Communication Services,66373336.0,USD,SPY,2023-01-12
...,...,...,...,...,...,...,...,...,...,...
503,Lincoln National Corp,LNC,53418710,2516378,0.014596,Financials,1715566.0,USD,SPY,2023-01-12
504,Fox Corporation Class B,FOX,35137L20,BJJMGY5,0.013262,Communication Services,1624680.0,USD,SPY,2023-01-12
505,DaVita Inc.,DVA,23918K10,2898087,0.013018,Health Care,616770.0,USD,SPY,2023-01-12
506,DISH Network Corporation Class A,DISH,25470M10,2303581,0.011113,Communication Services,2734767.0,USD,SPY,2023-01-12


In [384]:
# with open("./test_IShares_IVV_holdings_current_resp.csv", 'wb') as f:
#     f.write(fhun_.content)

ivv = ishares_listings[ishares_listings['ticker']=="IVV"].iloc[0]
ivv_hist = ISharesListings.retrieve_holdings_(ivv['ticker'], ivv['product_url'], date(year=2022, month=10, day=31))
ivv_now = ISharesListings.retrieve_holdings_(ivv['ticker'], ivv['product_url'], None)

In [397]:
ivv_now

,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,AAPL,APPLE INC,Information Technology,Equity,1.787452e+10,5.97,1.787452e+10,133981885.0,133.41,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
1,MSFT,MICROSOFT CORP,Information Technology,Equity,1.593026e+10,5.32,1.593026e+10,66790747.0,238.51,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,7.576094e+09,2.53,7.576094e+09,79522351.0,95.27,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,5.148520e+09,1.72,5.148520e+09,16143103.0,318.93,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2023-01-12
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,4.877005e+09,1.63,4.877005e+09,53517008.0,91.13,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,DISH,DISH NETWORK CORP CLASS A,Communication,Equity,3.387050e+07,0.01,3.387050e+07,2249037.0,15.06,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
505,NWS,NEWS CORP CLASS B,Communication,Equity,2.103345e+07,0.01,2.103345e+07,1066605.0,19.72,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
506,ETD_USD,ETD USD BALANCE WITH R93531,Cash and/or Derivatives,Cash,7.500000e+01,0.00,7.500000e+01,75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,2023-01-12
507,MARGIN_USD,FUTURES USD MARGIN BALANCE,Cash and/or Derivatives,Cash Collateral and Margins,-7.500000e+01,0.00,-7.500000e+01,-75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,2023-01-12


In [16]:
# from etf_scraper.api import query_listings, load_listings, SCRAPERS
# l = query_listings()
# l.to_csv('./listings.csv', index=False)

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:113: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  resp_df_.loc[:, "net_assets"] = resp_df_["net_assets"].apply(parse_net)
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "inception_date"] = ssga_web_data_df_[
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:318: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the value

In [46]:
l[(l['provider']=='Invesco')&(l['fund_name'].str.contains('QQQ'))]

,fund_name,inception_date,ticker,cusip,isin,asset_class,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider,benchmark,exchange
138,Invesco QQQ,1999-03-10,QQQ,46090E103,US46090E1038,NaN,NaN,NaN,NaN,https://www.invesco.com/us/financial-products/...,NaN,NaN,ETF,Invesco,NDX,NASDAQ
